In [1]:
%pip install llama-index


  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.
instructor 1.7.2 requires tenacity<10.0.0,>=9.0.0, but you have tenacity 8.5.0 which is incompatible.
langchain-google-vertexai 2.0.13 requires httpx<0.29.0,>=0.28.0, but you have httpx 0.27.2 which is incompatible.
langchain-pinecone 0.2.2 requires aiohttp<3.11,>=3.10, but you have aiohttp 3.11.11 which is incompatible.
llama-index-llms-huggingface 0.4.1 requires huggingface-hub<0.24.0,>=0.23.0, but you have huggingface-hub 0.28.1 which is incompatible.
open-webui 0.5.12 requires chromadb==0.6.2, but you have chromadb 0.5.23 which is incompatible.
open-webui 0.5.12 requires google-generativeai==0.7.2, but you have google-generativeai 0.8.4 which is incompatible.
open-webui 0.5.12 requires langchain==0.3.7, 